In [1]:
import numpy as np
from scipy.stats import qmc


In [2]:
def f(u):
    """
    u est un vecteur de taille d>=1 -> np.array
    """
    d = len(u)
    somme = np.sum(u)
    return np.cos(2*np.pi*(1/d * somme -0.5))

f(np.array([1,0.5,0.3,0.4]))

np.float64(0.9510565162951535)

# Question 1

$$\int_{[0,1]^d}^{}{f(u)du}  \simeq \frac{1}{N}\sum_{i=1}^{N}f(u_{(i)}) \text{ with } u_{(i)} \sim^{iid} U([0,1]^d)

In [3]:
# Monte Carlo standard
def monte_carlo_integration(d, N):
    samples = np.random.uniform(0, 1, (N, d))
    return np.mean(np.array([f(sample) for sample in samples]))

# Quasi-Monte Carlo (Sobol sequence)
def quasi_monte_carlo_integration(d, N):
    sobol = qmc.Sobol(d, scramble=True)
    samples = sobol.random(N)
    return np.mean(np.array([f(sample) for sample in samples]))

# Différentes valeurs de d et N
d_values = [1, 2, 3,5, 10, 100, 200]
N_values = [100, 1000, 10000, 20000]

# Stocker les résultats
results = {}

for d in d_values:
    results[d] = {}
    for N in N_values:
        mc_result = monte_carlo_integration(d, N)
        qmc_result = quasi_monte_carlo_integration(d, N)
        results[d][N] = (mc_result, qmc_result)

# Affichage des résultats
for d in results:
    print(f"Dimension d = {d}")
    for N in results[d]:
        mc, qmc_res = results[d][N]
        print(f"  N = {N}: Monte Carlo = {mc:.6f}, Quasi-Monte Carlo = {qmc_res:.6f}")
    print()

c:\Users\Célian\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_qmc.py:958: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


Dimension d = 1
  N = 100: Monte Carlo = 0.020523, Quasi-Monte Carlo = 0.000510
  N = 1000: Monte Carlo = 0.039154, Quasi-Monte Carlo = 0.000804
  N = 10000: Monte Carlo = 0.000285, Quasi-Monte Carlo = 0.000013
  N = 20000: Monte Carlo = -0.001585, Quasi-Monte Carlo = -0.000002

Dimension d = 2
  N = 100: Monte Carlo = 0.310479, Quasi-Monte Carlo = 0.416294
  N = 1000: Monte Carlo = 0.411838, Quasi-Monte Carlo = 0.404212
  N = 10000: Monte Carlo = 0.392037, Quasi-Monte Carlo = 0.405347
  N = 20000: Monte Carlo = 0.405744, Quasi-Monte Carlo = 0.405289

Dimension d = 3
  N = 100: Monte Carlo = 0.573709, Quasi-Monte Carlo = 0.581031
  N = 1000: Monte Carlo = 0.587956, Quasi-Monte Carlo = 0.565722
  N = 10000: Monte Carlo = 0.562386, Quasi-Monte Carlo = 0.565682
  N = 20000: Monte Carlo = 0.564922, Quasi-Monte Carlo = 0.565672

Dimension d = 5
  N = 100: Monte Carlo = 0.677379, Quasi-Monte Carlo = 0.710703
  N = 1000: Monte Carlo = 0.707595, Quasi-Monte Carlo = 0.715972
  N = 10000: Monte 

Quand $d \to \infty$, on remarque que $I \to 1$ \
Pour la dimension $d=2$ par exemple, on peut calculer l'intégrale double explicitement et on trouve $I = \frac{4}{\pi^2} \simeq 0.40528$ et ça nous permet de remarquer que QMC converge plus vite vers le bon résultat que MC.\
De même pour $d=1$, $I = 0$ par changement de variable, et on voit la vitesse de convergence plus rapide de QMC.

# Question 2

$$
e_{0,k} = \left\{ \left( \frac{2j_1 + 1}{2k}, \dots, \frac{2j_s + 1}{2k} \right) 
\text{ s.t. } (j_1, \dots, j_s) \in \{0, \dots, k-1\}^s \right\}
\ $$




$$\
\hat{I}_{1,k}(f) := \frac{1}{k^s} \sum_{c \in \mathcal{e}_0,k} f(c + U_c), \quad U_c \sim \mathcal{U} \left( \left[ -\frac{1}{2k}, \frac{1}{2k} \right]^s \right)
\


In [ ]:


def haber_estimator_1(f, k, s):
    """
    Estimateur de Haber (ordre 1) avec gestion mémoire efficace.

    Paramètres :
        f : fonction à intégrer
        k : nombre de subdivisions par dimension
        s : dimension de l'espace

    Retour :
        Estimation de l'intégrale de f sur [0,1]^s
    """
    total = 0.0
    count = 0
    shape = (k,) * s          # ex: (4, 4, 4)      

    # Pour chaque cellule du pavage (k^s cases)
    for idx in np.ndindex(*shape):
        #print(f"idx : {idx}")
        center = (np.array(idx) + 0.5) / k  # Centre de la cellule
        #print(f"centers : {center}")
        u = np.random.uniform(-0.5/k, 0.5/k, size=s)  # Perturbation uniforme
        total += f(center + u)
        count += 1

    return total / count



def haber_estimator_2(f, k, s):
    """
    Estimateur de Haber (ordre 2) avec boucle explicite pour réduire la mémoire.

    Paramètres :
        f : fonction à intégrer
        k : nombre de subdivisions par dimension
        s : dimension de l'espace

    Retour :
        Estimation de l'intégrale de f sur [0,1]^s
    """
    total = 0.0
    count = 0
    shape = (k,)*s
    for idx in np.ndindex(*shape):
        center = (np.array(idx) + 0.5) / k
        u = np.random.uniform(-0.5/k, 0.5/k, size=s)
        total += (f(center + u) + f(center - u)) / 2
        count += 1

    return total / count



# # Test function
# def test_function(x):
#     return np.cos(2 * np.pi * (np.mean(x, axis=1) - 0.5))



In [5]:
d_values = [1, 2, 3]
k_values = [10, 20, 50, 100]  # Number of subintervals per dimension


results = {}

for d in d_values:
    results[d] = {}
    for k in k_values:
        I1 = haber_estimator_1(f, k, d)
        I2 = haber_estimator_2(f, k, d)
        results[d][k] = (I1,I2)

# Affichage des résultats
for d in results:
    print(f"Dimension d = {d}")
    for k in results[d]:
        mc, qmc_res = results[d][k]
        print(f"k = {k}, d= {d},number of centers = {k**d}: Haber1 = {mc:.6f}, Haber2 = {qmc_res:.6f}")
    print()

Dimension d = 1
k = 10, d= 1,number of centers = 10: Haber1 = 0.028707, Haber2 = 0.000582
k = 20, d= 1,number of centers = 20: Haber1 = 0.022451, Haber2 = -0.001098
k = 50, d= 1,number of centers = 50: Haber1 = 0.001230, Haber2 = 0.000018
k = 100, d= 1,number of centers = 100: Haber1 = -0.000868, Haber2 = -0.000004

Dimension d = 2
k = 10, d= 2,number of centers = 100: Haber1 = 0.412161, Haber2 = 0.403614
k = 20, d= 2,number of centers = 400: Haber1 = 0.404865, Haber2 = 0.405311
k = 50, d= 2,number of centers = 2500: Haber1 = 0.405131, Haber2 = 0.405283
k = 100, d= 2,number of centers = 10000: Haber1 = 0.405277, Haber2 = 0.405284

Dimension d = 3
k = 10, d= 3,number of centers = 1000: Haber1 = 0.566761, Haber2 = 0.565630
k = 20, d= 3,number of centers = 8000: Haber1 = 0.565737, Haber2 = 0.565593
k = 50, d= 3,number of centers = 125000: Haber1 = 0.565608, Haber2 = 0.565595
k = 100, d= 3,number of centers = 1000000: Haber1 = 0.565594, Haber2 = 0.565596



Convergence d'$ Haber1$ et d'$Haber2$ vers la vraie valeur de $I$ beaucoup plus rapide que QMC et MC, par exemple en dimension 1 dès $N=20$ on est est déjà proche de 0 alors qu'il fallait attendre $N=100$ pour MC et QMC. 

On observe aussi une convergence plus rapide de la variance d'$Haber2$ grâce à la technique de réduction de variance (Antithetic Variable).

Plus k augmente, plus N augmente et plus l'estimateur d'Haber converge vers la valeur de l'intégrale.

In [6]:
d_values = [5,10]
k_values = [2,3,4]  # Number of subintervals per dimension


results = {}

for d in d_values:
    results[d] = {}
    for k in k_values:
        I1 = haber_estimator_1(f, k, d)
        I2 = haber_estimator_2(f, k, d)
        results[d][k] = (I1,I2)

# Affichage des résultats
for d in results:
    print(f"Dimension d = {d}")
    for k in results[d]:
        mc, qmc_res = results[d][k]
        print(f"  k = {k}, d= {d},number of centers = {k**d}: Haber1 = {mc:.6f}, Haber2 = {qmc_res:.6f}")
    print()

Dimension d = 5
  k = 2, d= 5,number of centers = 32: Haber1 = 0.781541, Haber2 = 0.717598
  k = 3, d= 5,number of centers = 243: Haber1 = 0.710370, Haber2 = 0.716102
  k = 4, d= 5,number of centers = 1024: Haber1 = 0.718984, Haber2 = 0.717322

Dimension d = 10
  k = 2, d= 10,number of centers = 1024: Haber1 = 0.842655, Haber2 = 0.847404
  k = 3, d= 10,number of centers = 59049: Haber1 = 0.847854, Haber2 = 0.847707
  k = 4, d= 10,number of centers = 1048576: Haber1 = 0.847896, Haber2 = 0.847857



In [16]:
d_values = [10]
k_values = [2,3]  # Number of subintervals per dimension


results = {}

for d in d_values:
    results[d] = {}
    for k in k_values:
        I1 = haber_estimator_1(f, k, d)
        I2 = haber_estimator_2(f, k, d)
        results[d][k] = (I1,I2)

# Affichage des résultats
for d in results:
    print(f"Dimension d = {d}")
    for k in results[d]:
        mc, qmc_res = results[d][k]
        print(f"  k = {k}, d= {d},number of centers = {k**d}: Haber1 = {mc:.6f}, Haber2 = {qmc_res:.6f}")
    print()

Dimension d = 10
  k = 2, d= 10,number of centers = 1024: Haber1 = 0.841889, Haber2 = 0.848720
  k = 3, d= 10,number of centers = 59049: Haber1 = 0.848227, Haber2 = 0.847982



Cependant, les estimateurs d'Haber ne sont pas adaptés quand la dimension explose ($d>>10$), parce que le nombre de centre $n=k^{d}$ est très important. Une voie d'exploration serait l'utilisation de rectangle.

# Question 3

### Step 1: Understanding the Approximate Distribution 

We analyze the distribution of the quantity:

$$
S_d = \frac{1}{d} \sum_{i=1}^{d} u_i
$$

when $( d )$ is large. Given that $( u_i \sim U(0,1) )$ (i.i.d.), we apply the **Central Limit Theorem (CLT)**. Since the expectation and variance of a uniform $( U(0,1) )$ variable are:

$$
\mathbb{E}[u_i] = \frac{1}{2}, \quad \text{Var}[u_i] = \frac{1}{12},
$$

we obtain:

$$
\mathbb{E}[S_d] = \frac{1}{d} \sum \mathbb{E}[u_i] = \frac{1}{2}
$$

$$
\text{Var}[S_d] = \frac{1}{d^2} \sum \text{Var}[u_i] = \frac{1}{12d}
$$
Then we have,

$$ \sqrt{d} \times \frac{S_{d} - \frac{1}{2}}{1/12} \simeq \mathcal{N} \left( 0, 1 \right)$$

Thus, for large $( d )$, we approximate:

$$
S_d \sim \mathcal{N} \left( \frac{1}{2}, \frac{1}{12d} \right)
$$



### Step  2 : Importance sampling

### Importance Sampling
$$\mathbb{E}_{x \sim P}[f(x)] = \mathbb{E}_{x \sim Q}\Big[f(x)\frac{P(x)}{Q(x)}\Big]$$
Which means  $\mathbb{E}_{x \sim P}[f(x)] \approx \frac{1}{n}\sum_{i=1}^nf(x_i)\frac{P(x_i)}{Q(x_i)}$ where $x_i$ are drawn from $Q$. This applies when $P$ and $Q$ are both normalized. For unnormalized case (not our case)   
$$\mathbb{E}_{x \sim P}[f(x)] \approx \frac{\sum_{i=1}^nf(x_i)\frac{P(x_i)}{Q(x_i)}}{\sum_{i=1}^n\frac{P(x_i)}{Q(x_i)}}$$  
Let the proposal distribution $Q(x)$ be a normal distribution.   


In [ ]:
import numpy as np
from scipy.stats import multivariate_normal, norm

def uniform_pdf(u,d):
    """
    Densité de probabilité pour une distribution uniforme sur [0,1]^d.
    Comme c'est uniforme, la densité est constante et vaut 1 sur l'espace.
    """
    if np.all((u >= 0) & (u <= 1)):  # Vérifie que u est bien dans [0,1]^d
        return 1
    else:
        return 0  # En dehors de [0,1]^d, la densité est nulle

def normal_pdf(u,d):
    """
    Densité de probabilité pour une loi normale multidimensionnelle de dimension d  : N(mu, Sigma)
    avec mu = (1/2, ..., 1/2) et Sigma = (1/12d) * I_d (matrice identité).
    """
    mu = np.full(d, 0.5)  # Vecteur moyenne [1/2, 1/2, ..., 1/2]
    sigma2 = 1 / (12 * d)  # Variance pour chaque variable
    sigma = np.eye(d) * sigma2  # Matrice de covariance diagonale

    return multivariate_normal.pdf(u, mean=mu, cov=sigma)


In [ ]:
def IS_MC(N,d, mu=1/2, sigma = None):
    if not sigma :
        sigma = 1/(12*d)
    somme = 0
    for i in range(N):
        sample = np.random.multivariate_normal(np.full(d, 0.5), np.eye(d) *1 / (12 * d))
        somme += f(sample) * (uniform_pdf(sample,d)/normal_pdf(sample,d))
    return somme/N

def IS_QMC(N,d, mu=1/2, sigma = None):
    if not sigma :
        sigma = 1/(12*d)
    """ Importance Sampling avec Quasi-Monte Carlo utilisant une séquence Sobol """
    sampler = qmc.Sobol(d, scramble=True)  # Génère des points Sobol dans [0,1]^d
    U = sampler.random(N)  # Matrice (N, d) avec des points uniformes
    # Transformation en loi normale N(0.5, 1/(12d)) avec la fonction quantile
    mu = 0.5
    sigma = np.sqrt(1 / (12 * d))
    X = norm.ppf(U, loc=mu, scale=sigma)  # Transforme U ~ U[0,1] en X ~ N(mu, sigma^2)
    


    # Importance Sampling
    somme = 0
    for x in X:
        somme += f(x) * (uniform_pdf(x,d) /normal_pdf(x, d))  
    return somme / len(X)


In [17]:
d_values = [1, 2, 3, 5]
N_values = [1000, 5000, 10000, 20000, 50000]  # Number of subintervals per dimension


results = {}

for d in d_values:
    results[d] = {}
    for N in N_values:
        I1 = IS_MC( N, d)
        I2 = IS_QMC( N, d)
        results[d][N] = (I1,I2)

# Affichage des résultats
for d in results:
    print(f"Dimension d = {d}")
    for k in results[d]:
        mc, qmc_res = results[d][k]
        print(f"  N = {k}: IS MC = {mc:.6f}, IS QMC = {qmc_res:.6f}")
    print()

Dimension d = 1
  N = 1000: IS MC = 0.033919, IS QMC = 0.002408
  N = 5000: IS MC = 0.008306, IS QMC = -0.000351
  N = 10000: IS MC = 0.023044, IS QMC = -0.000293
  N = 20000: IS MC = 0.004628, IS QMC = -0.000011
  N = 50000: IS MC = -0.000294, IS QMC = 0.000003

Dimension d = 2
  N = 1000: IS MC = 0.393438, IS QMC = 0.390166
  N = 5000: IS MC = 0.412456, IS QMC = 0.422232
  N = 10000: IS MC = 0.438745, IS QMC = 0.405676
  N = 20000: IS MC = 0.404754, IS QMC = 0.401684
  N = 50000: IS MC = 0.418245, IS QMC = 0.403757

Dimension d = 3
  N = 1000: IS MC = 0.494730, IS QMC = 0.603534
  N = 5000: IS MC = 0.534864, IS QMC = 0.569642
  N = 10000: IS MC = 0.531637, IS QMC = 0.601552
  N = 20000: IS MC = 0.500053, IS QMC = 0.565288
  N = 50000: IS MC = 0.597192, IS QMC = 0.551656

Dimension d = 5
  N = 1000: IS MC = 0.301219, IS QMC = 0.562003
  N = 5000: IS MC = 0.406448, IS QMC = 0.451629
  N = 10000: IS MC = 0.470515, IS QMC = 0.500818
  N = 20000: IS MC = 0.483931, IS QMC = 0.418338
  N = 

en d=5 ça devrait tendre vers 0.76 mais la comportement est quelque peu chaotique 

In [19]:
d_values = [20]
N_values = [10000]  # Number of subintervals per dimension


results = {}

for d in d_values:
    results[d] = {}
    for N in N_values:
        I1 = IS_MC( N, d)
        I2 = IS_QMC( N, d)
        results[d][N] = (I1,I2)

# Affichage des résultats
for d in results:
    print(f"Dimension d = {d}")
    for k in results[d]:
        mc, qmc_res = results[d][k]
        print(f"  N = {k}: IS MC = {mc:.6f}, IS QMC = {qmc_res:.6f}")
    print()

Dimension d = 20
  N = 10000: IS MC = 0.000000, IS QMC = 0.000000



Problème apparent !

# Problèmes et solutions pour l'Importance Sampling

## 1. Mauvais choix de la densité d'importance
L'Importance Sampling (IS) est très sensible au choix de la densité d'importance $g(x)$.  
Si $g(x)$ ne couvre pas bien les régions où $f(x)$ est significatif, les pondérations définies par :

$$ w(x) = \frac{p(x)}{g(x)} $$

peuvent devenir extrêmement variables, entraînant une grande variance de l'estimateur.  
En grande dimension, une loi normale centrée en $0.5$ peut mal représenter les régions importantes de $f(x)$, ce qui engendre une instabilité des résultats.

---

## 2. Problème de dégénérescence en grande dimension
Lorsque la dimension $d$ augmente, la plupart des échantillons issus de $g(x)$ se retrouvent éloignés des régions où $f(x)$ est significatif.  
Cela conduit à une situation où très peu d'échantillons contribuent réellement à l'intégrale, ce qui augmente la variance et rend l'estimateur instable.



---

## 3. Inefficacité des méthodes Quasi-Monte Carlo en grande dimension
Les méthodes Quasi-Monte Carlo (QMC) offrent un gain en convergence pour les dimensions faibles à modérées.  
Cependant, lorsque la dimension $d$ augmente, les séquences de faible discrépance comme Sobol ou Halton deviennent moins efficaces, car elles sont optimisées pour des espaces de faible dimension.


